In [1]:
import numpy as np
import os
import decimal

In [2]:
mainpath   = '/home/taylor/Documents/Research/spitzer/MEGA/'
planets = ['CoRoT-2b', 'HAT-P-7b', 'HAT-P-7b', 'HD149026b', 'HD149026b', 'KELT-16b', 'KELT-9b', 'MASCARA-1b', 'Qatar1b', 'Qatar1b', 'WASP-121b', 'WASP-121b', 'WASP-14b', 'WASP-14b', 'WASP-18b', 'WASP-18b', 'WASP-19b', 'WASP-19b', 'WASP-33b', 'WASP-33b', 'WASP-43b', 'WASP-43b', 'WASP-43b_repeatCh1', 'WASP-43b_repeatCh2', 'WASP-43b_repeat2Ch2', 'KELT-7b']
channels = ['ch2', 'ch1', 'ch2', 'ch1', 'ch2', 'ch2', 'ch2', 'ch2', 'ch1', 'ch2', 'ch1', 'ch2', 'ch1', 'ch2', 'ch1', 'ch2', 'ch1', 'ch2', 'ch1', 'ch2', 'ch1', 'ch2', 'ch1', 'ch2', 'ch2', 'ch2']

hide = ['WASP-121b', 'WASP-121b', 'WASP-43b_repeatCh1', 'WASP-43b_repeatCh2', 'WASP-43b_repeat2Ch2']
hide_channels = ['ch1', 'ch2', 'ch1', 'ch2', 'ch2']
for i in range(len(hide)):
    hide_ind = np.where(np.logical_and(np.array(planets)==hide[i], np.array(channels)==hide_channels[i]))[0][0]
    planets = np.delete(planets, hide_ind)
    channels = np.delete(channels, hide_ind)

planets = list(planets)
channels = list(channels)

twoLimits = False
onlyGoodFits = False
onlyBadFits = False

In [3]:
print(channels)

['ch2', 'ch1', 'ch2', 'ch1', 'ch2', 'ch2', 'ch2', 'ch2', 'ch1', 'ch2', 'ch1', 'ch2', 'ch1', 'ch2', 'ch1', 'ch2', 'ch1', 'ch2', 'ch1', 'ch2', 'ch2']


In [19]:
def roundToSigFigs(x, sigFigs=2):
    nDec = -int(np.floor(np.log10(np.abs(x))))+sigFigs-1
    rounded = np.round(x, nDec)
    if nDec <= 0:
        #remove decimals if this should be an integer
        rounded = int(rounded)
    output = str(rounded)
    if nDec > 1 and np.rint(rounded*10**nDec) % 10 == 0:
        #take care of the trailing zero issue
        output += '0'
    return nDec, output

def roundToDec(x, nDec=2):
    rounded = np.round(x, nDec)
    if nDec <= 0:
        #remove decimals if this should be an integer
        rounded = int(rounded)
    output = str(rounded)
    if nDec > 1 and np.rint(rounded*10**nDec) % 10 == 0:
        #take care of the trailing zero issue
        output += '0'
    return output

def makeTable1(planets, channels, twoLimits):
    output = r'''\begin{deluxetable*}{cc|cccc}[!htbp]
    \tabletypesize{\footnotesize}
    \tablewidth{\textwidth}
    \tablehead{\colhead{Data Set} & \colhead{Model} & \colhead{$\chi^2/n_{\rm data}$} & \colhead{$\sigma_F$ (ppm)} & \colhead{$\sigma_F/\sigma_{\rm limit}$} & \colhead{$F_{\rm day}/F_*$ (ppm)}}
    \decimals
    \startdata'''+'\n'
    for iterNum in range(len(planets)):
        channel = channels[iterNum]
        planet = planets[iterNum]

        with open(mainpath+planet+'/analysis/'+channel+'/bestPhOption.txt') as f:
            line = f.readline().strip()
            foldername = mainpath+line[line.find(planet):]+'/'

        modes = np.sort([x[0].split('/')[-1] for x in os.walk(mainpath+line[line.find(planet):])][1:])

        #find the best modes
        BICs = []
        nDatas = []
        for mode in modes:
            savepath   = foldername + mode + '/'+ 'ResultMCMC_'+mode+'_Params.npy'
            if not os.path.exists(savepath):
                BICs.append(np.inf)
            else:
                ResultMCMC = np.load(savepath)
                BICs.append(-2*ResultMCMC['evidenceB'][0])
                nDatas.append(ResultMCMC['chi2B'][0]/ResultMCMC['chi2datum'][0])
        BICs = np.array(BICs)
        nDatas = np.array(nDatas)

        try:
            bestBIC = BICs[np.argmin(BICs/nDatas)]
        except:
            # There are no analyses for this planet
            planetName = planet.split('_repeat')
            if len(planetName)>1:
                planetName = planetName[0]+' Repeat'+planetName[1][:-3]
            else:
                planetName = planetName[0]
            output += '        '+planetName+', '+channel+' & '
            output += '? & ? & ? & ? & ? '
            output += ' \\\\ % \n'
            continue

        for i in range(len(modes)):

            ######################## MAKE BESTFIT TABLE ########################

            if BICs[i] != bestBIC:
                continue

            mode = modes[i]
            savepath   = foldername + mode + '/' + 'ResultMCMC_'+mode+'_Params.npy'
            ResultMCMC = np.load(savepath)

            planetName = planet.split('_repeat')
            if len(planetName)>1:
                planetName = planetName[0]+' Repeat'+planetName[1][:-3]
            else:
                planetName = planetName[0]
            output += '        '+planetName+', '+channel+' & '

            if 'poly' in mode.lower():
                ind = mode.lower().find('poly')
                output += mode[ind:ind+5]
            elif 'bliss' in mode.lower():
                output += 'BLISS'
            elif 'pld' in mode.lower():
                output += 'PLD'
            elif 'gp' in mode.lower():
                output += 'GP'

            if 'tslope' in mode.lower():
                output += '*$f$(t)'

            if 'hside' in mode.lower():
                output += '*Step'

            if 'psfw' in mode.lower():
                output += ', PSFW'
            
            if 'trim' in mode.lower():
                output += ', Trim'

            output += ', '

            ind = mode.lower().find('_v')
            vorder = mode[ind+1:].split('_')[0][1:]

            if vorder == '1':
                output += '1st Order'
            elif vorder == '2':
                output += '2nd Order'

            if 'ellipse' in mode.lower() and 'offset' in mode.lower():
                output += ' + Rotated Ellipse & '
            elif 'ellipse' in mode.lower():
                output += ' + Ellipse & '
            else:
                output += ' & '


            output += '$'+str(roundToSigFigs(ResultMCMC['chi2datum'][0], sigFigs=4)[1])+'$ & '
            output += '$'+str(roundToDec(ResultMCMC['sigF'][0]*1e6, 0))+'$ & '
            try:
                output += '$'+str(roundToSigFigs(ResultMCMC['sigF'][0]*1e6/ResultMCMC['sigF_photon_ppm'][0], sigFigs=4)[1])+'$ & '
            except:
                output += '? & '

            val = ResultMCMC['fp']*1e6
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output += '$'+val+'\pm'+err+'$'

            output += ' \\\\ % \n'

    output += r'''    \enddata
\end{deluxetable*}'''

    return output
    
def makeTable2(planets, channels, twoLimits):
    output = r'''\begin{deluxetable*}{cc|cccccc}[!htbp]
    \tabletypesize{\footnotesize}
    \tablewidth{\textwidth}
    \tablehead{\colhead{Data Set} & \colhead{Model} & \colhead{$R_{\rm p}/R_{\rm *}$} & \colhead{$a/R_{\rm *}$} & \colhead{$i$ ($^{\circ}$)} & \colhead{$e$} & \colhead{$\omega$ ($^{\circ}$)}}
    \decimals
    \startdata'''+'\n'
    for iterNum in range(len(planets)):
        channel = channels[iterNum]
        planet = planets[iterNum]

        with open(mainpath+planet+'/analysis/'+channel+'/bestPhOption.txt') as f:
            line = f.readline().strip()
            foldername = mainpath+line[line.find(planet):]+'/'

        modes = np.sort([x[0].split('/')[-1] for x in os.walk(mainpath+line[line.find(planet):])][1:])

        #find the best modes
        BICs = []
        nDatas = []
        for mode in modes:
            savepath   = foldername + mode + '/'+ 'ResultMCMC_'+mode+'_Params.npy'
            if not os.path.exists(savepath):
                BICs.append(np.inf)
            else:
                ResultMCMC = np.load(savepath)
                BICs.append(-2*ResultMCMC['evidenceB'][0])
                nDatas.append(ResultMCMC['chi2B'][0]/ResultMCMC['chi2datum'][0])
        BICs = np.array(BICs)
        nDatas = np.array(nDatas)

        try:
            bestBIC = BICs[np.argmin(BICs/nDatas)]
        except:
            # There are no analyses for this planet
            planetName = planet.split('_repeat')
            if len(planetName)>1:
                planetName = planetName[0]+' Repeat'+planetName[1][:-3]
            else:
                planetName = planetName[0]
            output += '        '+planetName+', '+channel+' & '
            output += '? & ? & ? & ? & ? & ? '
            output += ' \\\\ % \n'
            continue

        for i in range(len(modes)):

            ######################## MAKE BESTFIT TABLE ########################

            if BICs[i] != bestBIC:
                continue

            mode = modes[i]
            savepath   = foldername + mode + '/' + 'ResultMCMC_'+mode+'_Params.npy'
            ResultMCMC = np.load(savepath)

            planetName = planet.split('_repeat')
            if len(planetName)>1:
                planetName = planetName[0]+' Repeat'+planetName[1][:-3]
            else:
                planetName = planetName[0]
            output += '        '+planetName+', '+channel+' & '

            if 'poly' in mode.lower():
                ind = mode.lower().find('poly')
                output += mode[ind:ind+5]
            elif 'bliss' in mode.lower():
                output += 'BLISS'
            elif 'pld' in mode.lower():
                output += 'PLD'
            elif 'gp' in mode.lower():
                output += 'GP'

            if 'tslope' in mode.lower():
                output += '*$f$(t)'

            if 'hside' in mode.lower():
                output += '*Step'

            if 'psfw' in mode.lower():
                output += ', PSFW'
            
            if 'trim' in mode.lower():
                output += ', Trim'

            output += ', '

            ind = mode.lower().find('_v')
            vorder = mode[ind+1:].split('_')[0][1:]

            if vorder == '1':
                output += '1st Order'
            elif vorder == '2':
                output += '2nd Order'

            if 'ellipse' in mode.lower() and 'offset' in mode.lower():
                output += ' + Rotated Ellipse & '
            elif 'ellipse' in mode.lower():
                output += ' + Ellipse & '
            else:
                output += ' & '


            #rp/rs, a/rs, per, inc, ecc, fday/fstar
            val = ResultMCMC['rp']
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output += '$'+val+'\pm'+err+'$ & '

            val = ResultMCMC['a']
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output += '$'+val+'\pm'+err+'$ & '

            val = ResultMCMC['inc']
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output += '$'+val+'\pm'+err+'$ & '

            ecosw = ResultMCMC['ecosw']
            esinw = ResultMCMC['ecosw']

            ecosw = np.random.normal(ecosw[0], np.mean(ecosw[1:3]), 100000)
            esinw = np.random.normal(esinw[0], np.mean(esinw[1:3]), 100000)

            ecc = np.sqrt(ecosw**2 + esinw**2)
            #longitude of periastron needs to be in degrees for batman!
            w   = np.arctan2(esinw, ecosw)*180./np.pi

            val = [np.median(ecc), np.std(ecc), np.std(ecc)]
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output += '$'+val+'\pm'+err+'$ & '

            if np.median(ecc)/np.std(ecc) < 3:
                output += '-- & '
            else:
                val = [np.median(w), np.std(w), np.std(w)]
                if twoLimits:
                    nDec1, err1 = roundToSigFigs(val[1])
                    nDec2, err2 = roundToSigFigs(val[2])
                    nDec = np.max([nDec1, nDec2])
                    val = roundToDec(val[0], nDec)
                    output += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
                else:
                    nDec, err = roundToSigFigs(np.mean(val[1:]))
                    val = roundToDec(val[0], nDec)
                    output += '$'+val+'\pm'+err+'$ & '

            output += ' \\\\ % \n'

    output += r'''    \enddata
\end{deluxetable*}'''

    return output

def makeTable3(planets, channels, twoLimits):
    output = r'''\begin{deluxetable*}{cc|ccccccc}[!htbp]
    \tabletypesize{\footnotesize}
    \tablewidth{\textwidth}
    \tablehead{\colhead{Data Set} & \colhead{Model} & \colhead{1st Amp.} & \colhead{1st Offset} & \colhead{2nd Amp.} & \colhead{2nd offset} & \colhead{$T_{\rm day}$ (K)} & \colhead{$T_{\rm night}$ (K)}}
    \decimals
    \startdata'''+'\n'
    for iterNum in range(len(planets)):
        channel = channels[iterNum]
        planet = planets[iterNum]

        with open(mainpath+planet+'/analysis/'+channel+'/bestPhOption.txt') as f:
            line = f.readline().strip()
            foldername = mainpath+line[line.find(planet):]+'/'

        modes = np.sort([x[0].split('/')[-1] for x in os.walk(mainpath+line[line.find(planet):])][1:])

        #find the best modes
        BICs = []
        nDatas = []
        for mode in modes:
            savepath   = foldername + mode + '/'+ 'ResultMCMC_'+mode+'_Params.npy'
            if not os.path.exists(savepath):
                BICs.append(np.inf)
            else:
                ResultMCMC = np.load(savepath)
                BICs.append(-2*ResultMCMC['evidenceB'][0])
                nDatas.append(ResultMCMC['chi2B'][0]/ResultMCMC['chi2datum'][0])
        BICs = np.array(BICs)
        nDatas = np.array(nDatas)

        try:
            bestBIC = BICs[np.argmin(BICs/nDatas)]
        except:
            # There are no analyses for this planet
            planetName = planet.split('_repeat')
            if len(planetName)>1:
                planetName = planetName[0]+' Repeat'+planetName[1][:-3]
            else:
                planetName = planetName[0]
            output += '        '+planetName+', '+channel+' & '
            output += '? & ? & ? & ? & ? & ? & ? '
            output += ' \\\\ % \n'
            continue


        for i in range(len(modes)):

            ######################## MAKE BESTFIT TABLE ########################

            if BICs[i] != bestBIC:
                continue

            mode = modes[i]
            savepath   = foldername + mode + '/' + 'ResultMCMC_'+mode+'_Params.npy'
            ResultMCMC = np.load(savepath)

            planetName = planet.split('_repeat')
            if len(planetName)>1:
                planetName = planetName[0]+' Repeat'+planetName[1][:-3]
            else:
                planetName = planetName[0]
            output += '        '+planetName+', '+channel+' & '

            if 'poly' in mode.lower():
                ind = mode.lower().find('poly')
                output += mode[ind:ind+5]
            elif 'bliss' in mode.lower():
                output += 'BLISS'
            elif 'pld' in mode.lower():
                output += 'PLD'
            elif 'gp' in mode.lower():
                output += 'GP'

            if 'tslope' in mode.lower():
                output += '*$f$(t)'

            if 'hside' in mode.lower():
                output += '*Step'

            if 'psfw' in mode.lower():
                output += ', PSFW'
            
            if 'trim' in mode.lower():
                output += ', Trim'

            output += ', '

            ind = mode.lower().find('_v')
            vorder = mode[ind+1:].split('_')[0][1:]

            if vorder == '1':
                output += '1st Order'
            elif vorder == '2':
                output += '2nd Order'

            if 'ellipse' in mode.lower() and 'offset' in mode.lower():
                output += ' + Rotated Ellipse & '
            elif 'ellipse' in mode.lower():
                output += ' + Ellipse & '
            else:
                output += ' & '

            if (planet=='WASP-33b' or planet=='WASP-14b' or planet=='WASP-18b' or planet=='WASP-19b') and channel=='ch1':
                print(planet, bestBIC)
                print(ResultMCMC['chi2B'][1]/ResultMCMC['chi2datum'][1])
                
            A = ResultMCMC['A']
            B = ResultMCMC['B']

            A = np.random.normal(A[0], np.mean(A[1:3]), 10000)
            B = np.random.normal(B[0], np.mean(B[1:3]), 10000)

            amplitude = 2*np.sqrt(A**2+B**2)
            offset = np.arctan2(B, A)*180/np.pi

            val = [np.median(amplitude), np.std(amplitude), np.std(amplitude)]
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output += '$'+val+'\pm'+err+'$ & '

            if np.median(offset) < 0:
                direction = 'E'
            else:
                direction = 'W'
            val = [np.abs(np.median(offset)), np.std(offset), np.std(offset)]
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output += '$'+val+'^{+'+err1+'}_{-'+err2+'}~^{\circ}$'+direction+' & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output += '$'+val+'\pm'+err+'~^{\circ}$'+direction+' & '


            if vorder == '1':
                output += '-- & -- & '
            else:
                C = ResultMCMC['C']
                D = ResultMCMC['D']

                C = np.random.normal(C[0], np.mean(C[1:3]), 10000)
                D = np.random.normal(D[0], np.mean(D[1:3]), 10000)

                amplitude = 2*np.sqrt(C**2+D**2)
                offset = np.arctan2(D, C)*180/np.pi/2

                val = [np.median(amplitude), np.std(amplitude), np.std(amplitude)]
                if twoLimits:
                    nDec1, err1 = roundToSigFigs(val[1])
                    nDec2, err2 = roundToSigFigs(val[2])
                    nDec = np.max([nDec1, nDec2])
                    val = roundToDec(val[0], nDec)
                    output += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
                else:
                    nDec, err = roundToSigFigs(np.mean(val[1:]))
                    val = roundToDec(val[0], nDec)
                    output += '$'+val+'\pm'+err+'$ & '

                if np.median(offset) < 0:
                    direction = 'E'
                else:
                    direction = 'W'
                val = [np.abs(np.median(offset)), np.std(offset), np.std(offset)]
                if twoLimits:
                    nDec1, err1 = roundToSigFigs(val[1])
                    nDec2, err2 = roundToSigFigs(val[2])
                    nDec = np.max([nDec1, nDec2])
                    val = roundToDec(val[0], nDec)
                    output += '$'+val+'^{+'+err1+'}_{-'+err2+'}~^{\circ}$'+direction+' & '
                else:
                    nDec, err = roundToSigFigs(np.mean(val[1:]))
                    val = roundToDec(val[0], nDec)
                    output += '$'+val+'\pm'+err+'~^{\circ}$'+direction+' & '


            val = ResultMCMC['tDay']
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output += '$'+val+'\pm'+err+'$ & '

            val = ResultMCMC['tNight']
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output += '$'+val+'\pm'+err+'$ & '



            output += ' \\\\ % \n'

    output += r'''    \enddata
\end{deluxetable*}'''

    return output

def makePosterTable(planets, channels, twoLimits):
    output = r'''\begin{deluxetable*}{cc|ccccccc}[!htbp]
    \tabletypesize{\footnotesize}
    \tablewidth{\textwidth}
    \tablehead{\colhead{Data Set} & \colhead{Model} & \colhead{$F_{\rm day}/F_{\rm *}$} & \colhead{Amplitude} & \colhead{Offset} & \colhead{$T_{\rm day}$ (K)} & \colhead{$T_{\rm night}$ (K)}}
    \decimals
    \startdata'''+'\n'
    for iterNum in range(len(planets)):
        channel = channels[iterNum]
        planet = planets[iterNum]

        with open(mainpath+planet+'/analysis/'+channel+'/bestPhOption.txt') as f:
            line = f.readline().strip()
            foldername = mainpath+line[line.find(planet):]+'/'

        modes = np.sort([x[0].split('/')[-1] for x in os.walk(mainpath+line[line.find(planet):])][1:])

        #find the best modes
        BICs = []
        nDatas = []
        for mode in modes:
            savepath   = foldername + mode + '/'+ 'ResultMCMC_'+mode+'_Params.npy'
            if not os.path.exists(savepath):
                BICs.append(np.inf)
            else:
                ResultMCMC = np.load(savepath)
                BICs.append(-2*ResultMCMC['evidenceB'][0])
                nDatas.append(ResultMCMC['chi2B'][0]/ResultMCMC['chi2datum'][0])
        BICs = np.array(BICs)
        nDatas = np.array(nDatas)

        try:
            bestBIC = BICs[np.argmin(BICs/nDatas)]
        except:
            # There are no analyses for this planet
            planetName = planet.split('_repeat')
            if len(planetName)>1:
                planetName = planetName[0]+' Repeat'+planetName[1][:-3]
            else:
                planetName = planetName[0]
            output += '        '+planetName+', '+channel+' & '
            output += '? & ? & ? & ? & ? & ? & ? '
            output += ' \\\\ % \n'
            continue


        for i in range(len(modes)):

            ######################## MAKE BESTFIT TABLE ########################

            if BICs[i] != bestBIC:
                continue

            mode = modes[i]
            savepath   = foldername + mode + '/' + 'ResultMCMC_'+mode+'_Params.npy'
            ResultMCMC = np.load(savepath)

            planetName = planet.split('_repeat')
            if len(planetName)>1:
                planetName = planetName[0]+' Repeat'+planetName[1][:-3]
            else:
                planetName = planetName[0]
            output += '        '+planetName+', '+channel+' & '

            if 'poly' in mode.lower():
                ind = mode.lower().find('poly')
                output += mode[ind:ind+5]
            elif 'bliss' in mode.lower():
                output += 'BLISS'
            elif 'pld' in mode.lower():
                output += 'PLD'
            elif 'gp' in mode.lower():
                output += 'GP'

            if 'tslope' in mode.lower():
                output += '*$f$(t)'

            if 'hside' in mode.lower():
                output += '*Step'

            if 'psfw' in mode.lower():
                output += ', PSFW'

            output += ', '

            ind = mode.lower().find('_v')
            vorder = mode[ind+1:].split('_')[0][1:]

            if vorder == '1':
                output += '1st Order'
            elif vorder == '2':
                output += '2nd Order'

            if 'ellipse' in mode.lower() and 'offset' in mode.lower():
                output += ' + Rotated Ellipse & '
            elif 'ellipse' in mode.lower():
                output += ' + Ellipse & '
            else:
                output += ' & '

            val = ResultMCMC['fp']
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1]*1e6)
                nDec2, err2 = roundToSigFigs(val[2]*1e6)
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0]*1e6, nDec)
                output += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:])*1e6)
                val = roundToDec(val[0]*1e6, nDec)
                output += '$'+val+'\pm'+err+'$ & '
            
            A = ResultMCMC['A']
            B = ResultMCMC['B']

            A = np.random.normal(A[0], np.mean(A[1:3]), 10000)
            B = np.random.normal(B[0], np.mean(B[1:3]), 10000)

            amplitude = 2*np.sqrt(A**2+B**2)
            offset = np.arctan2(B, A)*180/np.pi

            val = [np.median(amplitude), np.std(amplitude), np.std(amplitude)]
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output += '$'+val+'\pm'+err+'$ & '

            if np.median(offset) < 0:
                direction = 'E'
            else:
                direction = 'W'
            val = [np.abs(np.median(offset)), np.std(offset), np.std(offset)]
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output += '$'+val+'^{+'+err1+'}_{-'+err2+'}~^{\circ}$'+direction+' & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output += '$'+val+'\pm'+err+'~^{\circ}$'+direction+' & '

            val = ResultMCMC['tDay']
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output += '$'+val+'\pm'+err+'$ & '

            val = ResultMCMC['tNight']
            if twoLimits:
                nDec1, err1 = roundToSigFigs(val[1])
                nDec2, err2 = roundToSigFigs(val[2])
                nDec = np.max([nDec1, nDec2])
                val = roundToDec(val[0], nDec)
                output += '$'+val+'^{+'+err1+'}_{-'+err2+'}$ & '
            else:
                nDec, err = roundToSigFigs(np.mean(val[1:]))
                val = roundToDec(val[0], nDec)
                output += '$'+val+'\pm'+err+'$ & '



            output += ' \\\\ % \n'

    output += r'''    \enddata
\end{deluxetable*}'''

    return output

def trimPlanets(planets, channels, onlyGoodFits, onlyBadFits):
    new_planets = []
    new_channels = []
    for iterNum in range(len(planets)):
        channel = channels[iterNum]
        planet = planets[iterNum]

        with open(mainpath+planet+'/analysis/'+channel+'/bestPhOption.txt') as f:
            line = f.readline().strip()
            foldername = mainpath+line[line.find(planet):]+'/'

        modes = np.sort([x[0].split('/')[-1] for x in os.walk(mainpath+line[line.find(planet):])][1:])

        #find the best modes
        BICs = []
        for mode in modes:
            savepath   = foldername + mode + '/'+ 'ResultMCMC_'+mode+'_Params.npy'
            if not os.path.exists(savepath):
                BICs.append(np.inf)
            else:
                ResultMCMC = np.load(savepath)
                BICs.append(-2*ResultMCMC['evidenceB'][0])
        BICs = np.array(BICs)

        try:
            bestBIC = np.min(BICs)
        except:
            continue

        for i in range(len(modes)):

            ######################## MAKE BESTFIT TABLE ########################

            if BICs[i] != bestBIC:
                continue

            mode = modes[i]

            savepath   = foldername + mode + '/' + 'ResultMCMC_'+mode+'_Params.npy'
            if not os.path.exists(savepath):
                continue

            ResultMCMC = np.load(savepath)

            try:
                ratio = ResultMCMC['sigF'][0]*1e6/ResultMCMC['sigF_photon_ppm'][0]
            except:
                continue
            
            if onlyGoodFits and ratio <= 2:
                new_planets.append(planets[iterNum])
                new_channels.append(channels[iterNum])
            elif onlyBadFits and ratio > 2:
                print(planet, channel, 'is bad')
                new_planets.append(planets[iterNum])
                new_channels.append(channels[iterNum])
    return new_planets, new_channels

def addFileHeader():
    return r'''\documentclass[onecolumn]{aastex62}
\usepackage[utf8]{inputenc}
% \usepackage[margin=1in]{geometry}
\usepackage[section]{placeins}
\renewcommand{\floatpagefraction}{.99}%

\begin{document}
\pagenumbering{gobble}
'''
    

In [20]:
if onlyGoodFits or onlyBadFits:
    planets, channels = trimPlanets(planets, channels, onlyGoodFits, onlyBadFits)

In [21]:
output = addFileHeader()
output += makeTable1(planets, channels, twoLimits)
output += '\n\n\n\n'
output += makeTable2(planets, channels, twoLimits)
output += '\n\n\n\n'
output += makeTable3(planets, channels, twoLimits)
output += '\n\n\n\n'
output += '\end{document}'
print(output)

WASP-14b -18384.40730498006
1683.0
WASP-18b -37790.09134683156
3647.9999999999995
WASP-19b -6363.045056613066
651.0
WASP-33b -49092.43673968549
4623.0
\documentclass[onecolumn]{aastex62}
\usepackage[utf8]{inputenc}
% \usepackage[margin=1in]{geometry}
\usepackage[section]{placeins}
\renewcommand{\floatpagefraction}{.99}%

\begin{document}
\pagenumbering{gobble}
\begin{deluxetable*}{cc|cccc}[!htbp]
    \tabletypesize{\footnotesize}
    \tablewidth{\textwidth}
    \tablehead{\colhead{Data Set} & \colhead{Model} & \colhead{$\chi^2/n_{\rm data}$} & \colhead{$\sigma_F$ (ppm)} & \colhead{$\sigma_F/\sigma_{\rm limit}$} & \colhead{$F_{\rm day}/F_*$ (ppm)}}
    \decimals
    \startdata
        CoRoT-2b, ch2 & Poly3, 2nd Order & $0.9893$ & $1442$ & $1.987$ & $4200\pm150$ \\ % 
        HAT-P-7b, ch1 & Poly4, 2nd Order & $0.9792$ & $1300$ & $3.642$ & $1830\pm140$ \\ % 
        HAT-P-7b, ch2 & Poly3, 2nd Order & $0.9891$ & $885$ & $1.801$ & $2117\pm80$ \\ % 
        HD149026b, ch1 & Poly4, 2nd Order

In [7]:
planets_temp, channels_temp = trimPlanets(planets, channels, onlyGoodFits=True, onlyBadFits=False)
output = addFileHeader()
output += makePosterTable(planets_temp, channels_temp, twoLimits)
output += '\n\n\n\n'
output += '\end{document}'
print(output)

\documentclass[onecolumn]{aastex62}
\usepackage[utf8]{inputenc}
% \usepackage[margin=1in]{geometry}
\usepackage[section]{placeins}
\renewcommand{\floatpagefraction}{.99}%

\begin{document}
\pagenumbering{gobble}
\begin{deluxetable*}{cc|ccccccc}[!htbp]
    \tabletypesize{\footnotesize}
    \tablewidth{\textwidth}
    \tablehead{\colhead{Data Set} & \colhead{Model} & \colhead{$F_{\rm day}/F_{\rm *}$} & \colhead{Amplitude} & \colhead{Offset} & \colhead{$T_{\rm day}$ (K)} & \colhead{$T_{\rm night}$ (K)}}
    \decimals
    \startdata
        CoRoT-2b, ch2 & Poly3, 2nd Order & $4200\pm150$ & $0.893\pm0.051$ & $22.2\pm4.7~^{\circ}$W & $1813\pm34$ & $953\pm76$ &  \\ % 
        HAT-P-7b, ch2 & Poly3, 2nd Order & $2117\pm80$ & $0.964\pm0.071$ & $49.3\pm3.9~^{\circ}$W & $2851\pm71$ & $1699\pm100$ &  \\ % 
        HD149026b, ch2 & Poly5, 2nd Order & $356\pm32$ & $1.071\pm0.077$ & $44.7\pm4.2~^{\circ}$E & $1765\pm68$ & $1026\pm86$ &  \\ % 
        KELT-16b, ch2 & BLISS, 1st Order & $4890\pm220$ & $

In [8]:
'WASP-14', 1772/50*2.5

('WASP-14', 88.6)

In [9]:
'WASP-19', 693/50*3

('WASP-19', 41.58)

In [10]:
'WASP-33', 4866/50*2.5

('WASP-33', 243.29999999999998)

In [11]:
'WASP-18', 3800/50*2

('WASP-18', 152.0)

In [12]:
#Debugging tables
#chisq/datum, sigF/sigPhot, fday/fstar
#rp/rs, a/rs, per, inc, ecc, argp
#amp, offset, amp2, offset2, tday, tnight

In [13]:
# Presentation Table
#amp, offset, tday, tnight, sigF/sigPhot

In [14]:
for i in range(len(planets)):
    print(planets[i]+','+channels[i])

CoRoT-2b,ch2
HAT-P-7b,ch1
HAT-P-7b,ch2
HD149026b,ch1
HD149026b,ch2
KELT-16b,ch2
KELT-9b,ch2
MASCARA-1b,ch2
Qatar1b,ch1
Qatar1b,ch2
WASP-14b,ch1
WASP-14b,ch2
WASP-18b,ch1
WASP-18b,ch2
WASP-19b,ch1
WASP-19b,ch2
WASP-33b,ch1
WASP-33b,ch2
WASP-43b,ch1
WASP-43b,ch2
KELT-7b,ch2


In [15]:
#wasp18ch2, wasp19ch1, wasp33ch2, wasp43ch1